In [1]:
import pandas as pd
import numpy as np
import surprise
import apyori

In [2]:
data = surprise.Dataset.load_builtin("ml-100k")

Dataset ml-100k could not be found. Do you want to download it? [Y/n] Y
Trying to download dataset from http://files.grouplens.org/datasets/movielens/ml-100k.zip...
Done! Dataset ml-100k has been saved to C:\Users\Jysanuser/.surprise_data/ml-100k


In [9]:
data

In [3]:
trainingSet = data.build_full_trainset()

In [4]:
testSet = trainingSet.build_testset()

In [8]:
trainingSet

In [10]:
df = pd.DataFrame(data.__dict__['raw_ratings'], columns=['user_id','item_id','rating','timestamp'])

In [11]:
df.head()

,user_id,item_id,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [12]:
len(df)

100000

In [13]:
df.user_id.nunique()

943

In [15]:
trainingSet.n_users

943

In [18]:
testSet[:5]

[('196', '242', 3.0),
 ('196', '393', 4.0),
 ('196', '381', 4.0),
 ('196', '251', 3.0),
 ('196', '655', 5.0)]

In [20]:
trainingSet.n_items

1682

In [21]:
trainingSet.global_mean

3.52986

In [22]:
from surprise import KNNBasic

In [23]:
sim_options = {
    'name': 'cosine',
    'user_based': True
}

In [24]:
knn = KNNBasic(sim_options=sim_options)

In [25]:
knn.fit(trainingSet)

Computing the cosine similarity matrix...
Done computing similarity matrix.


In [35]:
predictions_knn = knn.test(testSet)

In [36]:
predictions_knn[0:4]

[Prediction(uid='196', iid='242', r_ui=3.0, est=4.144829856960016, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='196', iid='393', r_ui=4.0, est=3.624855349458667, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='196', iid='381', r_ui=4.0, est=3.728118778057887, details={'actual_k': 40, 'was_impossible': False}),
 Prediction(uid='196', iid='251', r_ui=3.0, est=4.344952376096852, details={'actual_k': 40, 'was_impossible': False})]

In [37]:
predictions_knn[8]

Prediction(uid='196', iid='663', r_ui=5.0, est=4.475491686108841, details={'actual_k': 40, 'was_impossible': False})

In [29]:
from surprise import SVD

In [30]:
svd=SVD()

In [31]:
svd.fit(trainingSet)

In [32]:
predictions=svd.test(testSet)

In [33]:
predictions[42]

Prediction(uid='186', iid='148', r_ui=4.0, est=3.8876727623365475, details={'was_impossible': False})

In [34]:
4.0 - 3.8876727623365475

0.11232723766345254

На платформе верный ответ 0.424. Делалось там под старый surprise

In [38]:
surprise.accuracy.rmse(predictions_knn) 

RMSE: 0.9012


0.9012175824620821

In [39]:
surprise.accuracy.rmse(predictions) 

RMSE: 0.6744


0.6743648330639991

Опять таки, на платформе ответ под старый surprise

In [41]:
df = pd.DataFrame(data.raw_ratings)
df.columns = ['user_id', 'movie_id', 'rating', 'timestamp']
df.head()

,user_id,movie_id,rating,timestamp
0,196,242,3.0,881250949
1,186,302,3.0,891717742
2,22,377,1.0,878887116
3,244,51,2.0,880606923
4,166,346,1.0,886397596


In [42]:
good = df[df['rating']>=4].groupby('user_id')['movie_id'].apply(lambda r: ' '.join([str(A) for A in r]))

In [43]:
good.head()

user_id
1      61 33 160 20 202 171 265 47 222 253 113 227 90...
10     16 486 611 7 100 488 285 504 289 340 505 489 6...
100    344 355 750 302 691 316 752 313 879 300 328 12...
101    405 121 24 284 50 237 181 924 257 742 255 117 ...
102    195 307 89 202 186 183 98 208 510 50 101 588 1...
Name: movie_id, dtype: object

In [44]:
type(good)

pandas.core.series.Series

In [45]:
good[100]

'4 435 153 310 258 319 887 211 325 202 8'

In [46]:
association_rules = apyori.apriori(good.apply(lambda r: r.split(' ')), 
                                   min_support=0.2, 
                                   min_confidence=0.3,
                                   min_lift=2, 
                                   min_length=2)

In [48]:
asr_df = pd.DataFrame(columns = ['from', 'to', 'confidence', 'support', 'lift'])

In [49]:
for item in association_rules:
    pair = item[0] 
    items = [x for x in pair]
    asr_df.loc[len(asr_df), :] =  ' '.join(list(item[2][0][1])), \
                                  ' '.join(list(item[2][0][0])),\
                                  item[2][0][2], item[1], item[2][0][3]

In [50]:
asr_df

,from,to,confidence,support,lift
0,174,172,0.819113,0.254777,2.21725
1,174,173,0.762097,0.200637,2.06292
2,195,174,0.548851,0.20276,2.38257
3,204,174,0.543103,0.200637,2.17704
4,210,174,0.563218,0.208068,2.25767
5,79,174,0.591954,0.218684,2.1122
6,174 181,172,0.651877,0.20276,2.6583
7,50 174,172,0.78157,0.2431,2.51276
8,50 181,172,0.750853,0.233546,2.02666
9,50 98,174,0.577586,0.213376,2.12534


In [51]:
len(asr_df)

10